# INTRODUCTION

1. [Read Datas](#1)
2. [Poverty Rate Of Each State](#2)
3. [Most Common 15 Name Or Surname Of Killed People](#3)
4. [High School Graduation Rate Of The Population That Is Older Than 25 In States](#4)
5. [Percentage Of State's Population According To Races That Are Black,White,Native American, Asian And Hispanic](#5)
6. [High School Graduation Rate Vs Poverty Rate Of Each State](#6)
    
    * [Race Rates According In Kill Data ](#7)
    * [Different Types Of Visualizations About Those Who Have Been Killed](#8)



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import seaborn as sns 
import matplotlib.pyplot as plt
from collections import Counter

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id = "1" ></a>   
# 1. Read Datas (Veri Okuma)

In [ ]:
median_house_hold_in_come = pd.read_csv('/kaggle/input/fatal-police-shootings-in-the-us/MedianHouseholdIncome2015.csv', encoding="windows-1252")

percentage_people_below_poverty_level = pd.read_csv('/kaggle/input/fatal-police-shootings-in-the-us/PercentagePeopleBelowPovertyLevel.csv', encoding="windows-1252")

percent_over_25_completed_highSchool = pd.read_csv('/kaggle/input/fatal-police-shootings-in-the-us/PercentOver25CompletedHighSchool.csv', encoding="windows-1252")

share_race_city = pd.read_csv('/kaggle/input/fatal-police-shootings-in-the-us/ShareRaceByCity.csv', encoding="windows-1252")

kill = pd.read_csv('/kaggle/input/fatal-police-shootings-in-the-us/PoliceKillingsUS.csv', encoding="windows-1252")

In [ ]:
percentage_people_below_poverty_level.head()

In [ ]:
percentage_people_below_poverty_level.info()
# Geographic Area = Eyaletlerin isimleri
# City = Eyaletlerin şehrileri
# poverty_rate = yoksuluk oranları

In [ ]:
percentage_people_below_poverty_level.poverty_rate.value_counts()  
# yoksulluk oranı bilinmeyen şehirlerimiz var
# yoksulluk oranı sıfır olan şehirlerimiz var

In [ ]:
percentage_people_below_poverty_level["Geographic Area"].unique()
# .unique() ile orjnal sıralamada verileri döndürdük

<a id = "2" ></a>   
# 2. Poverty rate of each state
# (Her eyaletin yoksulluk oranı)

In [ ]:
percentage_people_below_poverty_level.poverty_rate.replace(["-"],0.0,inplace = True)
# '-' olan 201 tane ifademiz var. Sıfır değeri ile değiştirdik

percentage_people_below_poverty_level.poverty_rate = percentage_people_below_poverty_level.poverty_rate.astype(float)
# poverty_rate(yoksulluk oranı) sütunu object veri tipinde , biz onun float olmasını istedik

area_list = list(percentage_people_below_poverty_level["Geographic Area"].unique()) 
# Geographic Area sütununu liste haline getirdik

area_poverty_ratio = []

for i in area_list:
    x = percentage_people_below_poverty_level[percentage_people_below_poverty_level["Geographic Area"] == i]
    area_poverty_rate = sum(x.poverty_rate)/len(x) # x.poverty_rate toplamını kelime sayısına böl
    # yukarıdaki kodu verimizin sıralı olmasını büyükten küçüğe sıralı olmasını istediğimiz için yaptık
    area_poverty_ratio.append(area_poverty_rate)  
    
data = pd.DataFrame({"area_list":area_list,"area_poverty_ratio":area_poverty_ratio})
# data değişkenimize iki adet yeni oluşturudğumuz kolon atadık  
new_index = (data["area_poverty_ratio"].sort_values(ascending=False)).index.values
# new_index değişkenimize "area_poverty_ratio" kolonuna göre büyükten küçüğe sıralama yaptırdık
sorted_data = data.reindex(new_index)
# reindex: isteğe bağlı doldurma ile yeni dizine uygun hale getirir



plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data["area_list"],y=sorted_data["area_poverty_ratio"])
# x ekseni için eyalet isimleri , y ekseni için eyaletlerin değerleri yer alır
plt.xticks(rotation= 0) # her sütun altındaki yazının duruş şekli
plt.xlabel("States")
plt.ylabel("Poverty Rate")
plt.title("Poverty Rate Given States")
plt.show()

<a id = "3" ></a>   
# 3. Most Common 15 Name or Surname of Killed People
# (En yaygın Ölen 15 kişinin Adı veya Soyadı)

In [ ]:
kill.head()

* manner_of_death	= nasıl öldürüldüğü
* armed = neyle öldürüldüğü
* age= ölenin yaşı 
* gander = öelenin cinsiyeti
* race = ölenin ırkı 
* city = yaşadığı şehir 
* state = bulunduğu eyalet
* signs_of_mental_illness = psikolojik hastalığı var mı
* threat_level = tehlike seviyesi 
* flee = ölen kişi ölmeden önce kaçmış mı
* body_camera = vücut kamerası var mı 

In [ ]:
kill.name.value_counts()
# tk tk ismiyle  bir veri 49 kere tekrar etmiş
# gerçek bir isimlendirme olmadığı için analize dahil etmiyoruz

In [ ]:
# most common 15 name or surname of killed people 
# En yaygın 15 isim veya öldürülen kişilerin soyadı

separate = kill.name[kill.name != "TK TK"].str.split()
# split(): default değeri boşluklardan kelime kelime ayırma işlemi yapar
a,b = zip(*separate) 
# * ile anzip yaparak ayırma işlemi yaptık
# a isim , b soyisim için
name_list = a+b
# a+b yaparak isim ve soyisi sütunlarını alt alta birleştirdik
name_count = Counter(name_list)
#counter fonksiyonu ile hangi isimden kaç adet kullanıldığını analiz ettik
most_common_names = name_count.most_common(15)
# .most_common(15)  : en çok kullanılan 15 isimi çıkartıyoruz
x,y = zip(*most_common_names)
# *most_common_names değişkeni içindedki verileride anzip yapıyorum ve x,y bçlüyorum
x,y = list(x),list(y)
# x ve y yi liste haline getiriyorum

plt.figure(figsize=(15,10))
sns.barplot(x=x,y=y,palette = sns.cubehelix_palette(len(x),reverse = True))
# cubehelix_palette: Bu, doğrusal olarak azalan (veya artan) parlaklığa sahip bir renk haritası oluşturur. 
# Bu, siyah beyaz yazdırıldığında veya renk körü biri tarafından görüntülendiğinde bilgilerin korunacağı anlamına gelir
# Renkler default olarak geliyor 
# len(x): len fonksiyonu ile x değeri içindeki eleman sayısı kadar dikdörtgen oluştu
plt.xlabel("Name or Surname of killed people")
plt.ylabel("Frequency")
plt.title("most common 15 name or surname of killed people")
plt.show()

<a id = "4" ></a>   

# 4. High School Graduation Rate Of The Population That Is Older Than 25 In States
# (Eyalette 25 Yaş Üstü Nüfusun Lise Mezuniyet Oranı)
   


In [ ]:
percent_over_25_completed_highSchool.head()
# percent_completed_hs :  liseden mezun olma oranı 

In [ ]:
percent_over_25_completed_highSchool.percent_completed_hs.value_counts()
# 197 adet '-' var, bunları değiştireceğiz ilk olarak

In [ ]:
# High school graduation rate of the population that is older than 25 in states

percent_over_25_completed_highSchool.percent_completed_hs.replace(["-"],0.0,inplace = True)
percent_over_25_completed_highSchool.percent_completed_hs = percent_over_25_completed_highSchool.percent_completed_hs.astype(float)
area_list = list(percent_over_25_completed_highSchool["Geographic Area"].unique())
area_highschool = []
for i in  area_list:
    x = percent_over_25_completed_highSchool[percent_over_25_completed_highSchool["Geographic Area"] == i]
    area_highschool_rate = sum(x.percent_completed_hs)/len(x)
    # oranı topla , kelimenin uzunluğuna böl 
    area_highschool.append(area_highschool_rate)
    
# sorting    
data = pd.DataFrame({"area_list":area_list,"area_highschool_ratio":area_highschool})
new_index = (data["area_highschool_ratio"].sort_values(ascending= False)).index.values
sorted_data2 = data.reindex(new_index)
# visualizaztion
plt.figure(figsize=(15,10))
ax = sns.barplot(x=sorted_data2["area_list"], y=sorted_data2["area_highschool_ratio"])
plt.xticks(rotation=0)
plt.xlabel("States")
plt.ylabel("High school Graduate Rate")
plt.title("Percentage of Given State's Population Above 25 that Has Graduated High School")
plt.show()

<a id = "5" ></a>   
# 5. Percentage of State's Population According To Races That Are Black,White,Native American, Asian And Hispanic
# (Siyah, Beyaz, Kızılderili, Asyalı ve İspanyol Irklara Göre Eyalet Nüfus Yüzdesi )


In [ ]:
share_race_city.head()


In [ ]:
share_race_city.info()

In [ ]:
share_race_city["share_asian"].value_counts()

In [ ]:
# Percentage of state's population according to races that are black,white,native american, asian and hispanic
# Siyah, beyaz, Kızılderili, Asya ve İspanyol ırklarına göre eyalet nüfusunun yüzdesi 

share_race_city.replace(["-"],0.0,inplace=True)
share_race_city.replace(["(X)"],0.0,inplace=True)
share_race_city.loc[:,["share_white","share_black","share_native_american","share_asian","share_hispanic"]] = share_race_city.loc[:,['share_white','share_black','share_native_american','share_asian','share_hispanic']].astype(float)
area_list= list(share_race_city["Geographic area"].unique())
share_white = []
share_black = []
share_native_american = []
share_asian = []
share_hispanic = []

for i in area_list:
    x = share_race_city[share_race_city["Geographic area"] == i]
    share_white.append(sum(x.share_white)/len(x))  # 
    share_black.append(sum(x.share_black)/len(x))
    share_native_american.append(sum(x.share_native_american)/len(x))
    share_asian.append(sum(x.share_asian)/len(x))
    share_hispanic.append(sum(x.share_hispanic)/len(x))

# visualization

f,ax = plt.subplots(figsize = (9,15))
sns.barplot(x=share_white,y=area_list,color="green",alpha=0.5,label="White")  #  alpha : saydamlık 
sns.barplot(x=share_black,y=area_list,color="blue",alpha=0.7,label="African American")
sns.barplot(x=share_native_american,y=area_list,color="cyan",alpha=0.6,label="Native American")
sns.barplot(x=share_asian,y=area_list,color="yellow",alpha=0.6,label="Asian")
sns.barplot(x=share_hispanic,y=area_list,color="red",alpha=0.6,label="Hispanic")

ax.legend(loc="lower right",frameon = True) # sağ altta kutunun nerde konumlandırılacağını ve saydam olmaması için ise True yapıyoruz
ax.set(xlabel="Percentage of Races",ylabel="States",title="Percentage of State's Population According to Races")
plt.show()

<a id = "6" ></a>   
# 6. High School Graduation Rate vs Poverty Rate Of Each State
# (Lise Mezuniyet Oranı İle Her Eyaletin Yoksulluk Oranı)


In [ ]:
sorted_data2

### Point Plot


In [ ]:
# high school graduation rate vs Poverty rate of each state
# lise mezuniyet oranı ve her eyaletin yoksulluk oranı 

# aşağıdaki kod parçasında sayıları normalize ettik 
# neden normalize ettik ; çünkü sayılar arasında büyük farklar var ise görselleştirme sırasında anlamsız görüntü oluşabilir
#tüm değereleri içinded bulunan en büyük değere bölerek 0 ile 1 arasında sayı değerleri elde ettik
# bunları karşılaştırmak daha iyi bir sonuç yakalamamıza neden olacaktır
sorted_data["area_poverty_ratio"] = sorted_data["area_poverty_ratio"]/max(sorted_data["area_poverty_ratio"])
sorted_data2["area_highschool_ratio"] = sorted_data2["area_highschool_ratio"]/max(sorted_data2["area_highschool_ratio"])

# concat ile iki sütunu yan yana birleştirdim
data = pd.concat([sorted_data,sorted_data2["area_highschool_ratio"]],axis=1) 
data.sort_values("area_poverty_ratio",inplace= True)


# visualize

f,ax = plt.subplots(figsize=(20,10))

# data = data ; birleştirdiğimiz iki sütunun değişkeninden bahsediyoruz , sns.pointplot bunu kendisi anlıyor
# x ve y için hangi kolonları kullanacağımızı belirtiyoruz
sns.pointplot(x="area_list",y="area_poverty_ratio",data=data,color="lime",alpha=0.8)
sns.pointplot(x="area_list",y="area_highschool_ratio",data=data,color="red",alpha=0.8)


plt.text(40,0.6,"high_school graduate ratio",color="red",fontsize=17,style="italic")
plt.text(40,0.55,"poverty ratio",color="lime",fontsize=18,style="italic")
plt.xlabel("States",fontsize = 15,color="blue")
plt.ylabel("Values",fontsize = 15,color="blue")
plt.title("High School Graduate  VS  Poverty Rate",fontsize=20,color="blue")
plt.grid()

## Join Plot

In [ ]:
g = sns.jointplot(data=data,x="area_poverty_ratio", y="area_highschool_ratio",kind="kde",height=5)

# data.area_poverty_ratio x ekseni
# data.area_highschool_ratio y eksini
# height=7 şeklin büyüklüğü
plt.show()

In [ ]:
g = sns.jointplot("area_poverty_ratio","area_highschool_ratio",data=data,height=5,ratio=3,color="r")
# ratio = 3 ; 
plt.show()

## Lm Plot

In [ ]:
# Visualization of high school graduation rate vs Poverty rate of each state with different style of seaborn code
# Show the results of a linear regression within each dataset
sns.lmplot(x="area_poverty_ratio",y="area_highschool_ratio",data=data)
plt.show()

## Kde Plot

In [ ]:
sns.kdeplot(data.area_poverty_ratio,data.area_highschool_ratio, shade= True,cut=3)
plt.show()

## Violin Plot

In [ ]:
# Show each distribution with both violins and points
# Use cubehelix to get a custom sequential palette

pal = sns.cubehelix_palette(2,rot=-.5,dark=.3)
sns.violinplot(data=data,palette=pal,inner="points")
plt.show()

## Heatmap

In [ ]:
data.corr()

In [ ]:
f,ax = plt.subplots(figsize = (5,5))
sns.heatmap(data.corr(), annot = True, linewidths=0.5, linecolor = "red", fmt = ".1f", ax=ax)
plt.show()

## Pair Plot

In [ ]:
sns.pairplot(data)
plt.show()

<a id = "7" ></a>   
# 7. Race Rates According In Kill Data 
# Irkların Öldürülme Oranı

## Pie Chart

In [ ]:
kill.head(10)

In [ ]:
kill.race.head(15) 
# race: ölen kişinin ırkı

In [ ]:
kill.race.value_counts()
# W = white 
# B = black
# H = hispanic
# A = asian
# N = native
# O = other

In [ ]:
# Race rates according in kill data 
# ırakların öldürülme oranı 

kill.race.dropna(inplace=True)
labels = kill.race.value_counts().index  # index : ırkların belirtildiği harfler
colors = ["grey","blue","red","yellow","green","brown"] 
explode = [0,0,0,0,0,0]
sizes = kill.race.value_counts().values #values : ırkların aldığı değerler

# visual
plt.figure(figsize=(7,7))
plt.pie(sizes,explode=explode,labels=labels,colors=colors,autopct="%1.1f%%")
# autopct="%1.1f%%" bir tane ondalıklı kısımdan göster demek
plt.title("Killed People According to Races",color="blue",fontsize=15)
plt.legend()
plt.show()

<a id="8" ></a>
# 8. Different Types Of Visualizations About Those Who Have Been Killed
# Öldürülenlerle ilgili farklı görselleştirme türleri

## Box Plot

In [ ]:
kill.head()

In [ ]:
kill.manner_of_death.unique()

In [ ]:
sns.boxplot(x="gender", y="age",hue="manner_of_death",data=kill,palette="PRGn")
plt.show()

## Swarm Plot 

In [ ]:
kill.head()

In [ ]:
sns.swarmplot(x="gender",y="age",hue="manner_of_death",data=kill)
plt.show()

## Count Plot


In [ ]:
kill.gender.value_counts()

In [ ]:
# kill properties
# Manner of death
# ölenlerin cinsiyeti
sns.countplot(kill.gender)

plt.title("gender",color="blue",fontsize =15)
plt.show()

In [ ]:
# kill weapon 
# kullanılan silahlar
armed = kill.armed.value_counts()
print(armed)

In [ ]:
# the way people are killed
# insanların öldürülme şekli
plt.figure(figsize=(10,7))
sns.barplot(x=armed[:7].index,y=armed[:7].values)
plt.ylabel("Number of Weapon")
plt.xlabel("Weapon Types")
plt.title("Kill weapon", color="blue",fontsize=15)
plt.show()

In [ ]:
# age of killed people 
# öldürülen insanların yaşı
above25 = ["above25" if i >= 25 else "below25"  for i in kill.age]
df = pd.DataFrame({"age":above25})
sns.countplot(x=df.age)
plt.ylabel("number of killed people")
plt.title("Age of killed people",color = "blue", fontsize=15)
plt.show()

In [ ]:
# Race of killed people 
# Öldürülen insanların ırkı
sns.countplot(data=kill , x="race")
plt.title("Race of killed people",color ="blue",fontsize=15)
plt.show()

In [ ]:
# most dangerous cities
# en tehlikeli şehirler
city = kill.city.value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=city[:12].index,y=city[:12].values)
plt.xticks(rotation=45)
plt.title("Most dangerous cities",color="blue",fontsize=15)
plt.show()

In [ ]:
# most dangerous states
# en tehlikeli eyalet

state = kill.state.value_counts()
plt.figure(figsize=(10,7))
sns.barplot(x=state[:20].index,y=state[:20].values)
plt.title("Most dangerous state",color="blue",fontsize=15)
plt.show()

In [ ]:
# Having mental ilness or not for killed people 
# Ölen insanlar için akıl hastalığına sahip olmak veya olmamak

sns.countplot(kill.signs_of_mental_illness)
plt.xlabel("Mental illness")
plt.ylabel("Number of Mental illness")
plt.title("Having mental illness or not",color="blue",fontsize=15)
plt.show()

In [ ]:
# Threat types
# Tehdit türleri

sns.countplot(kill.threat_level)
plt.xlabel("Threat Types")
plt.title("Threat types",color="blue",fontsize=15)
plt.show()

In [ ]:
# Flee Types 
# Kaçma TÜrleri

sns.countplot(kill.flee)
plt.xlabel("Flee Types")
plt.title("Flee types",color="blue",fontsize=15)
plt.show()

In [ ]:
# Having body cameras or not for police
# Polis için vücut kamerası olan veya olmayan

sns.countplot(kill.body_camera)
plt.xlabel("Having Body Cameras")
plt.title("Having body cameras or not on Police",color="blue",fontsize=15)
plt.show()

In [ ]:
# Kill number from states in kill data

sta = kill.state.value_counts().index[:10]
sns.barplot(x=sta,y=kill.state.value_counts().values[:10])
plt.show()